In [351]:
from z3 import *
from itertools import combinations, permutations, product

In [352]:
board=[[5,8,6,3,4],0,[7,1,6,1,8],0,[9,7,3,2,1],1]

In [353]:
board1=[[9,0,3,4,2],2,[7,0,7,9,4],0,[3,9,4,5,8],2,[3,4,1,0,9],1,[5,1,5,4,5],2,[1,2,5,3,1],1]

In [354]:
board12=[[9,0,3,4,2],2,2,
         [7,0,7,9,4],0,2,
         [3,9,4,5,8],2,2,
         [3,4,1,0,9],1,2,
         [5,1,5,4,5],2,0,
         [1,2,5,3,1],1,2] 

In [355]:
board=[[5,6,1,6,1,8,5,6,5,0,5,1,8,2,9,3],2,[3,8,4,7,4,3,9,6,4,7,2,9,3,0,4,7],1,[5,8,5,5,4,6,2,9,4,0,8,1,0,5,8,7],3,[9,7,4,2,8,5,5,5,0,7,0,6,8,3,5,3],3,[4,2,9,6,8,4,9,6,4,3,6,0,7,5,4,3],3,[3,1,7,4,2,4,8,4,3,9,4,6,5,8,5,8],1,[4,5,1,3,5,5,9,0,9,4,1,4,6,1,1,7],2,[7,8,9,0,9,7,1,5,4,8,9,0,8,0,6,7],3,[8,1,5,7,3,5,6,3,4,4,1,1,8,4,8,3],1,[2,6,1,5,2,5,0,7,4,4,3,8,6,8,9,9],2,[8,6,9,0,0,9,5,8,5,1,5,2,6,2,5,4],3,[6,3,7,5,7,1,1,9,1,5,0,7,7,0,5,0],1,[6,9,1,3,8,5,9,1,7,3,1,2,1,3,6,0],1,[6,4,4,2,8,8,9,0,5,5,0,4,2,7,6,8],2,[2,3,2,1,3,8,6,1,0,4,3,0,3,8,4,5],0,[2,3,2,6,5,0,9,4,7,1,2,7,1,4,4,8],2,[5,2,5,1,5,8,3,3,7,9,6,4,4,3,2,2],2,[1,7,4,8,2,7,0,4,7,6,7,5,8,2,7,6],3,[4,8,9,5,7,2,2,6,5,2,1,9,0,3,0,6],1,[3,0,4,1,6,3,1,1,1,7,2,2,4,6,3,5],3,[1,8,4,1,2,3,6,4,5,4,3,2,4,5,8,9],3,[2,6,5,9,8,6,2,6,3,7,3,1,6,8,6,7],2]

board2 = [
[1,1,2,2],0,1,
[3,4,4,1],1,1,
[5,3,5,1],1,1,
[6,6,3,1],0,1,
[2,4,5,3],1,3
]

In [356]:
def mastermind1_aux(B):
    # px10 matrix of booleans where p is the number of positions/length of the sequence
    P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(B[0])+1)]
    s = Solver()
    # One digit must be true per position:
    for pos in range(len(P)):
        s.add(Or(P[pos]))
        # No more than one digit can be true per position:
        for digit in range(10):
            for i in range(digit+1,10):
                s.add(Not(And(P[pos][digit],P[pos][i])))
    for i in range(0,len(B),2):
        b = [] # List of relevant booleans for the current line of the board
        aux = []
        for j in range(len(B[i])):
            b.append(P[j][B[i][j]])
        if B[i+1] == 0:
            s.add(Not(Or(b))) # These variables must necessarily be false since no position was correct
        else:
            comb = list(combinations(b,B[i+1])) # List of tuples of length k, where k is the number of correct positions
            for tuples in comb:
                other = [ x for x in b if x not in tuples ] # Complement set of current tuple
                aux.append(And(And(tuples),Not((Or(other))))) # If current tuple is true, all other numbers are false
            s.add(Or(aux)) # One of the previous conditions must hold from the set of all tuples
    print('Finding a solution ...')
    count = 0
    #found = False
    if s.check() == sat:
        m = s.model()
        solution = []
        # Search boolean matrix for trues to present solution:
        for pos in range(len(P)):
            for digit in range(10):
                if m[P[pos][digit]]:
                    solution.append(digit)
        count += 1
        print(f'Solution {count}: {solution}')
     #   found = True
        return [s,P,solution]
        
    else:
        print('No solutions found')

def mastermind1(B):
    return mastermind1_aux(B)[2]
def unique_solution1(B):
    R = mastermind1_aux(B)
    S = R[0]
    P = R[1]
    solution = R[2]
    # Deny the solution given by mastermind1 in the model to see if it's unique:
    for pos in range(len(P)):
            S.add(Not(P[pos][solution[pos]]))
    return S.check() == unsat
    

#unique_solution1(board1)   

'''def mastermind2_aux(B):
    # px10 matrix of booleans where p is the number of positions/length of the sequence
    P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(B[0])+1)]
    s = Solver()
    # One digit must be true per position:
    for pos in range(len(P)):
        s.add(Or(P[pos]))
        # No more than one digit can be true per position:
        for digit in range(10):
            for i in range(digit+1,10):
                s.add(Not(And(P[pos][digit],P[pos][i])))
    for i in range(0,len(B),3):
        b = [] # List of relevant booleans for the current line of the board

        aux_black = []
        aux_white = []
        for j in range(len(B[i])):
            b.append(P[j][B[i][j]])
        if B[i+1] == 0:
            s.add(Not(Or(b))) # These variables must necessarily be false since no position was correct
        else:
            comb_black = list(combinations(b,B[i + 1])) # List of tuples of length k, where k is the number of correct positions
            comb_white = list(combinations(list(enumerate(B[i])),B[i + 2]))

            for tuples in comb_black:
                other = [ x for x in b if x not in tuples ] # Complement set of current tuple
                aux_black.append(And(And(tuples),Not((Or(other))))) # If current tuple is true, all other numbers are false
            
            s.add(Or(aux_black))
            pos = [i for i in range(len(B[i]))]
            for tuples in comb_white:
                lt = len(tuples)
                ns = [tuples[i][1] for i in range (lt)]
                other = [ x for x in B[i] if x not in [tuples[j][1] for j in range(lt)] ] # Complement set of current tuple
                other = [P[j][x] for x in other]
                adds = list(product(pos,ns))
                aux_white.append(And(And(Or([P[x[0]][x[1]] for x in adds])),Not(Or([P[x[0]][x[1]] for x in tuples]))))# If current tuple is true, all other numbers are false

        
            s.add(Or(aux_white)) # One of the previous conditions must hold from the set of all tuples
    
    print('Finding all solutions...')
    count = 0
    found = False
    while s.check() == sat:
        m = s.model()
        solution = []
        # Search boolean matrix for trues to present solution:
        for pos in range(len(P)):
            for digit in range(10):
                if m[P[pos][digit]]:
                    solution.append(digit)
        count += 1
        print(f'Solution {count}: {solution}')
        found = True
        # Deny this solution in the model to search for others:
        for pos in range(len(P)):
            s.add(Not(P[pos][solution[pos]]))
    if found == False:
        print('No solutions found')'''

def mastermind2_aux(B):
    # px10 matrix of booleans where p is the number of positions/length of the sequence
    P = [[Bool(f'p_{p}_{n}') for n in range(10)] for p in range (1,len(B[0])+1)]
    s = Solver()
    # One digit must be true per position:
    for pos in range(len(P)):
        s.add(Or(P[pos]))
    # No more than one digit can be true per position:
        for digit in range(10):
            for i in range(digit+1,10):
                s.add(Not(And(P[pos][digit],P[pos][i])))

    for i in range(0,len(B),3):

        comb_black = list(combinations(list(enumerate(B[i])),B[i + 1])) # List of tuples of length k, where k is the number of correct positions
        
        
        aux = [] 
        posit = [x for x in range(len(B[i]))]
        #print(adds_white)
        for b in comb_black:
            #aux.append(And([P[x[0]][x[1]] for x in b]))
            bval = [x[1] for x in b]
            bpos = [x[0] for x in b]
            W_avlbl = [x for x in list(enumerate(B[i])) if x not in b]
            comb_white = list(combinations(W_avlbl,B[i + 2]))
            
            aux1 = []
            for w in comb_white:
                wval = [x[1] for x in w]
                wpos = [x[0] for x in w]
                pos_avlbl = [x for x in posit if x not in bpos]
                pos_for_w = list(permutations(pos_avlbl,B[i + 2]))
                wh = []
                for x in pos_for_w:
                    ps = []
                    for k in range(len(w)):
                        ps.append((x[k],w[k][1]))
                    wh.append(ps)

                other = [x for x in w]
                other_n = [x for x in B[i] if x not in bval + wval]
                for p in posit:
                    for n in other_n:
                        other.append((p,n))
                
                for w1 in wh:
                    #aux1.append(And(And([P[x[0]][x[1]] for x in w1]), Not(Or([P[x[0]][x[1]] for x in other]))))
                    aux1.append(And([P[x[0]][x[1]] for x in w1]))
                    aux=aux
                
            aux.append(And(And([P[x[0]][x[1]] for x in b]), Or(aux1)))
            #aux.append(And([P[x[0]][x[1]] for x in b]))
            #aux.append(Or(aux1))

     
        s.add(Or(aux))
    print('Finding all solutions...')
    count = 0
    found = False
    print(s.check())
    while s.check() == sat:
        m = s.model()
        solution = []
        # Search boolean matrix f   print(adds)or trues to present solution:
        for pos in range(len(P)):
            for digit in range(10):
                if m[P[pos][digit]]:
                    solution.append(digit)
        count += 1
        print(f'Solution {count}: {solution}')
        found = True
        # Deny this solution in the model to search for others:
        for pos in range(len(P)):
            s.add(Not(P[pos][solution[pos]]))
    if found == False:
        print('No solutions found')

mastermind2_aux(board12)

Finding all solutions...
sat
Solution 1: [3, 9, 5, 4, 2]


In [357]:
#rabiscos e apêndic(it)es:
for i in range(0,len(board),3):
    b = [] # List of relevant booleans for the current line of the board
    aux = []
    for j in range(len(board[i])):
        b.append(P[j][board[i][j]])
    if board[i+1]==0:
        if board[i+2] == 0:
            # None of the proposed numbers can be true, regardless of position:
            for row in range(len(P)):
                aux.append(P[row][board[i][row]])
            s.add(Not(Or(aux)))
        else:
            combswap = list(combinations(list(enumerate(board[i])),board[i+2]))
            #s.add(Not(Or(b)))
            for tuples in combswap:
                keep=[]
                swap=[]
                possible=list(permutations(list(enumerate(board[i])),board[i+2]))
                for k in range(len(tuples)):
                    for move in range(len(possible)-1,-1,-1):
                        if possible[move][k] == tuples[k]:
                            possible.pop(move)
                    keep.append(P[tuples[k][0]][tuples[k][1]])
                for groups in possible:
                    for h in range(len(groups)):
                        swap.append(P[groups[h][0]][tuples[h][1]])
                aux.append(And(Not(Or(keep)),And(swap)))
            s.add(Or(aux))
s.check()






#    elif board[i+2] == 0:
#        comb = list(combinations(b,board[i+1]))
#        for tuples in comb:
#            other = [ x for x in b if x not in tuples ] # Complement set of current tuple
#            aux.append(And(And(tuples),Not((Or(other))))) # If current tuple is true, all other numbers are false
#        s.add(Or(aux)) # One of the previous conditions must hold from the set of all tuples
#    else:
#        combc = list(combinations(list(enumerate(board[i])),board[i+1]))
#        combs = list(combinations(list(enumerate(board[i])),board[i+2]))
#        print('C:',combc)
#        print('S:',combs)

NameError: name 'P' is not defined